In [ ]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)


### First we define our Message object

Whatever structure we want for messages in our Agent framework.

In [ ]:
# Let's have a simple one!

# 加上 @dataclass 後，Python 會自動為類別產生初始化方法 (__init__)、表示方法 (__repr__)、比較方法 (__eq__) 等
# 可以更方便地建立和操作資料物件。
@dataclass
class Message:
    content: str


### Now we define our Agent

A subclass of RoutedAgent.

- RoutedAgent 的功能是根據消息類型智能地路由和處理來自用戶的消息，以便提供高效的對話互動和反應。

Every Agent has an **Agent ID** which has 2 components:  
`agent.id.type` describes the kind of agent it is  
`agent.id.key` gives it its unique identifier

Any method with the `@message_handler` decorated will have the opportunity to receive messages.


In [ ]:
# RoutedAgent 是一個基於路由的代理類別，允許根據消息的內容或類型將消息路由到不同的處理方法。
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        # 這裡的 "Simple" 是這個代理的 type 名稱
        super().__init__("Simple")

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.")
        

### OK let's create a Standalone runtime and register our agent type

In [ ]:

runtime = SingleThreadedAgentRuntime()

# simple_agent 將作為 SimpleAgent Agent 的 type 名稱，在 SingleThreadedAgentRuntime 中註冊
# lambda 用來延遲初始化 SimpleAgent 實例
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())  

# 註冊後，系統會自動給予 "simple_agent" 這個 type 一個預設的 key，通常是 "default"。

### Alright! Let's start a runtime and send a message

In [ ]:
runtime.start()

In [ ]:
# AgentId 用於識別和管理不同的代理實例， simple_agent 是 Agent 的 type， default 是實體的 key
agent_id = AgentId("simple_agent", "default")

# runtime 會將消息發送給指定的代理（simple_agent），並等待其回應
response = await runtime.send_message(Message("Well hi there!"), agent_id)
print(">>>", response.content)

In [ ]:
await runtime.stop()
await runtime.close()

### OK Now let's do something more interesting

We'll use an AgentChat Assistant!

In [ ]:
# MyLLMAgent 類別，繼承自 RoutedAgent，負責處理 LLM 相關訊息
class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        # 呼叫父類別初始化，設定 agent 類型為 "LLMAgent"
        super().__init__("LLMAgent")
        # 建立 OpenAIChatCompletionClient，指定模型為 gpt-4o-mini
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        # 建立 AssistantAgent，並將 model_client 傳入，作為委派物件
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        # 印出收到的訊息內容，方便除錯與追蹤
        print(f"{self.id.type} received message: {message.content}")
        # 將收到的 Message 轉換為 TextMessage，來源標記為 "user"
        text_message = TextMessage(content=message.content, source="user")
        # 使用委派的 AssistantAgent 處理訊息，並取得回覆
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        # 取得回覆內容
        reply = response.chat_message.content
        # 印出回覆內容，方便追蹤
        print(f"{self.id.type} responded: {reply}")
        # 將回覆包裝成 Message 物件並回傳
        return Message(content=reply)

In [ ]:
from autogen_core import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

In [ ]:
runtime.start()  # Start processing messages in the background.
response = await runtime.send_message(Message("Hi there!"), AgentId("LLMAgent", "default"))
# print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

In [ ]:
await runtime.stop()
await runtime.close()

### OK now let's show this at work - let's have 3 agents interact!

In [ ]:
from autogen_ext.models.ollama import OllamaChatCompletionClient


class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="llama3.2", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [ ]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)


In [ ]:
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [ ]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

In [ ]:
await runtime.stop()
await runtime.close()